In [0]:
SANDBOX_NAME = 'fesc' # Sandbox Name
DATA_PATH = "/data/sandboxes/"+SANDBOX_NAME+"/data/"

In [0]:
from google.colab import drive

In [4]:
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive




# Agrupando

Podemos agrupar información de nuestras estructuras de datos de forma muy sencilla mediante el método `groupby`. Normalmente se sigue una estrategia de separar-aplicar-combinar (*split-apply-combine*). Lo que se hace es separar los datos iniciales en grupos de interés, sobre cada grupo se aplica cierta funcionalidad y el resultado se combina en una nueva estructura de datos.

In [0]:
import pandas as pd



swim100m.csv Source: ‘Statistical Modeling: A Fresh Approach’

In [7]:
# data = spark.read.csv('/content/drive/My Drive/Colab Notebooks/Data/swim100m.csv', header=True, inferSchema=True).toPandas()

#Local reading

data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Data/swim100m.csv')
data.head()

,year,time,sex
0,1905,65.8,M
1,1908,65.6,M
2,1910,62.8,M
3,1912,61.6,M
4,1918,61.4,M


In [8]:
data.shape

(62, 3)



Agrupamos por sexo

In [9]:
grouped_sex = data.groupby(by='sex')
grouped_sex



El número de agrupaciones por sexo (len(grouped_sex)) es igual al número de elementos únicos de esa categoria

In [10]:
data['sex'].nunique() == len(grouped_sex)

True

In [11]:
data['sex'].nunique()

2

In [12]:
len(grouped_sex)

2



Números de elementos de cada grupo

In [13]:
grouped_sex.size()

sex
F    31
M    31
dtype: int64

In [14]:
data['sex'].value_counts()

F    31
M    31
Name: sex, dtype: int64



Devuelve el primer elemento de cada grupo

In [15]:
grouped_sex.first()

,year,time
sex,,
F,1908,95.0
M,1905,65.8




Devuelve un diccionario con los grupos creados y las rtiquetas (registros) que pertecen a cada grupo.

In [16]:
grouped_sex.groups

{'F': Int64Index([31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47,
             48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61],
            dtype='int64'),
 'M': Int64Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
             17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30],
            dtype='int64')}

In [17]:
grouped_sex.groups['M']

Int64Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
            17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30],
           dtype='int64')

In [19]:
data[data['sex'] == 'M'].index

Int64Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
            17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30],
           dtype='int64')



Mediante get_group podemos seleccionar uno de los grupos y devolvernos su información como un dataframe

In [20]:
grouped_sex.get_group('M').head()

,time,year
0,65.8,1905
1,65.6,1908
2,62.8,1910
3,61.6,1912
4,61.4,1918




Una vez realizada la agrupación que nos interesa podemos aplicar una transformación sobre alguna de las variables restantes.



En este caso calculamos la media del año por sexo

In [21]:
a = data.groupby('sex')['year'].mean()
print(a)

sex
F    1950.677419
M    1953.612903
Name: year, dtype: float64




Se puede iterar sobre el resultado obtenido con `groupby` (devuelve una tupla). El primer item es el valor de las columnas y el segundo es el `DataFrame` filtrado:

In [0]:
for k, v in data.groupby('sex'):
    print(k, ':', v.mean(), '\n')



También se pueden agrupar los datos respecto a más de una columna. El resultado son tuplas anidadas, por ejemplo:

In [22]:
# auto = spark.read.csv(DATA_PATH+'Auto.csv', header=True, inferSchema=True).toPandas()

#Local reading

auto = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Data/Auto.csv')
auto.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
0,18.0,8,307.0,130,3504,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165,3693,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150,3436,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150,3433,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140,3449,10.5,70,1,ford torino


In [23]:
auto.shape

(397, 9)

In [24]:
auto.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 397 entries, 0 to 396
Data columns (total 9 columns):
mpg             397 non-null float64
cylinders       397 non-null int64
displacement    397 non-null float64
horsepower      397 non-null object
weight          397 non-null int64
acceleration    397 non-null float64
year            397 non-null int64
origin          397 non-null int64
name            397 non-null object
dtypes: float64(3), int64(4), object(2)
memory usage: 28.0+ KB


In [27]:
auto_grouped = auto.groupby(['horsepower', 'cylinders']).mean()
auto_grouped.head()

mpg  displacement       weight  acceleration  \
horsepower cylinders                                                       
100        3          23.700000     70.000000  2420.000000     12.500000   
           4          32.900000    119.000000  2615.000000     14.800000   
           6          18.433333    236.133333  3228.066667     16.133333   
102        4          20.000000    130.000000  3150.000000     15.700000   
103        5          20.300000    131.000000  2830.000000     15.900000   

                      year  origin  
horsepower cylinders                
100        3          80.0     3.0  
           4          81.0     3.0  
           6          73.8     1.0  
102        4          76.0     2.0  
103        5          78.0     2.0

In [37]:
auto_grouped.loc['100',3]['year']

80.0



Otra opción para trabajar con datos agrupados en `Pandas` es utilizar la función agg(), que nos permite aplicar varias funciones sobre una agrupación.

In [0]:
#fortune = spark.read.csv(DATA_PATH+'fortune1000.csv', header=True,inferSchema=True).toPandas().set_index('Rank', inplace=True)

#Local reading

fortune = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Data/fortune1000.csv', index_col='Rank')

In [40]:
fortune.head()

,Company,Sector,Industry,Location,Revenue,Profits,Employees
Rank,,,,,,,
1,Walmart,Retailing,General Merchandisers,"Bentonville, AR",482130,14694,2300000
2,Exxon Mobil,Energy,Petroleum Refining,"Irving, TX",246204,16150,75600
3,Apple,Technology,"Computers, Office Equipment","Cupertino, CA",233715,53394,110000
4,Berkshire Hathaway,Financials,Insurance: Property and Casualty (Stock),"Omaha, NE",210821,24083,331000
5,McKesson,Health Care,Wholesalers: Health Care,"San Francisco, CA",181241,1476,70400




Agrupamos por sector y realizamos operaciones sobre cada grupo y varios campos.

In [47]:
sectors = fortune.groupby(by='Sector')
fortune_by_sector = sectors.agg({'Revenue':['sum','mean'], 'Profits':'sum','Employees':'mean'}) 
fortune_by_sector.reset_index().head()

Sector  Revenue               Profits     Employees
                            sum          mean     sum          mean
0  Aerospace & Defense   357940  17897.000000   28742  48402.850000
1              Apparel    95968   6397.866667    8236  23093.133333
2    Business Services   272195   5337.156863   28227  26687.254902
3            Chemicals   243897   8129.900000   22628  15455.033333
4               Energy  1517809  12441.057377  -73447   9745.303279

In [46]:
fortune[fortune['Company'] == 'Walmart']['Profits'] > fortune_by_sector.loc['Retailing'][('Profits', 'sum')]

Rank
1    False
Name: Profits, dtype: bool

In [49]:
fortune_by_sector.loc['Retailing'][('Profits', 'sum')]

47830.0

In [0]:
help(sectors.agg)



Otra forma de agrupar la información de un dataframe es a través de anidar índices, dando lugar a una **multiIndex dataframe**

In [0]:
bigmac = spark.read.csv(DATA_PATH+'bigmac.csv', 
                        header=True, inferSchema=True).toPandas()
bigmac['Date'] = pd.to_datetime(bigmac['Date'], infer_datetime_format=True)

#Local reading

# bigmac = pd.read_csv('../../data/bigmac.csv', parse_dates= ['Date'])
bigmac.head(3)

In [0]:
bigmac.set_index(keys = ['Date','Country'], inplace=True) # Creates a multindex DF
bigmac.sort_index(inplace= True)
bigmac



El acceso a los datos se realiza pasando la tupla de valores correspondiente al índice

In [0]:
bigmac.loc[('2010-01-01','Argentina'),'Price in US Dollars']



# Ejercicios



## Ejercicio 1 

Dado el fichero de datos datasets/u.user



- a. Cargue el fichero en un dataframe (indexe por 'user_id') y estudie las estructura de los datos.
- b. ¿Cúal es la edad media por profesión?.
- c. Para cada profesión, ¿cúal es la edad mínima y máxima?.
- d. Para cada combinación de profesión y género, calcule la edad media.
- e. Para cada profesión, calcule el porcentaje de mujeres y hombres



**a.** Cargue el fichero en un dataframe (indexe por 'user_id') y estudie las estructura de los datos.

In [54]:
users = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Data/u.user', sep='|', index_col='user_id')
users.head()

,age,gender,occupation,zip_code
user_id,,,,
1,24,M,technician,85711
2,53,F,other,94043
3,23,M,writer,32067
4,24,M,technician,43537
5,33,F,other,15213


In [56]:
users.shape

(943, 4)

In [57]:
users.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 943 entries, 1 to 943
Data columns (total 4 columns):
age           943 non-null int64
gender        943 non-null object
occupation    943 non-null object
zip_code      943 non-null object
dtypes: int64(1), object(3)
memory usage: 36.8+ KB


In [58]:
users.dtypes

age            int64
gender        object
occupation    object
zip_code      object
dtype: object

In [59]:
users.describe(include='all')

,age,gender,occupation,zip_code
count,943.000000,943,943,943
unique,NaN,2,21,795
top,NaN,M,student,55414
freq,NaN,670,196,9
mean,34.051962,NaN,NaN,NaN
std,12.192740,NaN,NaN,NaN
min,7.000000,NaN,NaN,NaN
25%,25.000000,NaN,NaN,NaN
50%,31.000000,NaN,NaN,NaN
75%,43.000000,NaN,NaN,NaN




**b.** ¿Cúal es la edad media por profesión?

In [70]:
import numpy as np

users_grouped = users.groupby('occupation')
for g, v in users_grouped:
    print(g, np.percentile(v['age'],q=[.25, .5, .75]))
    

administrator [21.195 21.39  21.585]
artist [19.0675 19.135  19.2025]
doctor [28.015 28.03  28.045]
educator [23.47 23.94 24.41]
engineer [22. 22. 22.]
entertainment [15.085 15.17  15.255]
executive [22.2325 22.465  22.6975]
healthcare [22.15 22.3  22.45]
homemaker [20.045 20.09  20.135]
lawyer [21.165 21.33  21.495]
librarian [23.25 23.5  23.75]
marketing [24. 24. 24.]
none [11.04 11.08 11.12]
other [14.3 15.6 16.9]
programmer [20.1625 20.325  20.4875]
retired [51.195 51.39  51.585]
salesman [18.0825 18.165  18.2475]
scientist [23.075 23.15  23.225]
student [ 8.4625  9.925  11.3875]
technician [21.065 21.13  21.195]
writer [18. 18. 18.]


In [74]:
import numpy as np

users.groupby(by='occupation').agg(({'age':'mean'}))

,age
occupation,
administrator,38.746835
artist,31.392857
doctor,43.571429
educator,42.010526
engineer,36.388060
entertainment,29.222222
executive,38.718750
healthcare,41.562500
homemaker,32.571429




**c.** Para cada profesión, ¿cúal es la edad mínima y máxima?

In [75]:
users_by_occ = users.groupby(by='occupation')
users_by_occ.agg({'age':['max','min']})

age    
              max min
occupation           
administrator  70  21
artist         48  19
doctor         64  28
educator       63  23
engineer       70  22
entertainment  50  15
executive      69  22
healthcare     62  22
homemaker      50  20
lawyer         53  21
librarian      69  23
marketing      55  24
none           55  11
other          64  13
programmer     63  20
retired        73  51
salesman       66  18
scientist      55  23
student        42   7
technician     55  21
writer         60  18



**d.** Para cada combinación de profesión y género, calcule la edad media

In [0]:
users_by_occ_gender = users.groupby(by=['occupation','gender']).agg({'age':['mean','min','max']})



**e**. Para cada profesión, calcule el porcentaje de mujeres y hombres.



Calculamos el total por profesiones

In [79]:
users_by_occ_gender.mean()

age  mean    36.286706
     min     23.975610
     max     52.804878
dtype: float64

In [0]:
porcentaje = [x/users.shape[0] * 100 for x in users.groupby(['occupation','gender']).size()]

In [81]:
porcentaje

[3.8176033934252387,
 4.559915164369035,
 1.378579003181336,
 1.5906680805938493,
 0.7423117709437964,
 2.757158006362672,
 7.317073170731707,
 0.2120890774125133,
 6.892895015906681,
 0.2120890774125133,
 1.6967126193001063,
 0.3181336161187699,
 3.0752916224814424,
 1.166489925768823,
 0.5302226935312832,
 0.6362672322375398,
 0.10604453870625664,
 0.2120890774125133,
 1.0604453870625663,
 3.0752916224814424,
 2.332979851537646,
 1.0604453870625663,
 1.6967126193001063,
 0.4241781548250266,
 0.5302226935312832,
 3.8176033934252387,
 7.317073170731707,
 0.6362672322375398,
 6.362672322375397,
 0.10604453870625664,
 1.378579003181336,
 0.3181336161187699,
 0.9544008483563097,
 0.3181336161187699,
 2.9692470837751856,
 6.362672322375397,
 14.422057264050903,
 0.10604453870625664,
 2.757158006362672,
 2.014846235418876,
 2.757158006362672]

In [0]:
users_by_occ_gender = users.groupby(['occupation','gender']).size()
users_by_occ = users.groupby(['occupation']).size()

In [90]:
users_by_occ_gender / users_by_occ * 100

occupation     gender
administrator  F          45.569620
               M          54.430380
artist         F          46.428571
               M          53.571429
doctor         M         100.000000
educator       F          27.368421
               M          72.631579
engineer       F           2.985075
               M          97.014925
entertainment  F          11.111111
               M          88.888889
executive      F           9.375000
               M          90.625000
healthcare     F          68.750000
               M          31.250000
homemaker      F          85.714286
               M          14.285714
lawyer         F          16.666667
               M          83.333333
librarian      F          56.862745
               M          43.137255
marketing      F          38.461538
               M          61.538462
none           F          44.444444
               M          55.555556
other          F          34.285714
               M          65.714286
progra



Creamos un DF agrupado por occupation/gender y calculamos el número por gender

In [0]:
# Respuesta aqui



Dividimos el segundo DF (occupation/gender/count) entre el primero (count por profesion)

In [0]:
# Respuesta aqui



## Ejercicio 2 

Dados los siguientes datos:

raw_data = {'regiment': ['Nighthawks', 'Nighthawks', 'Nighthawks', 'Nighthawks', 'Dragoons', 'Dragoons', 'Dragoons', 'Dragoons', 'Scouts', 'Scouts', 'Scouts', 'Scouts'], 
        'company': ['1st', '1st', '2nd', '2nd', '1st', '1st', '2nd', '2nd','1st', '1st', '2nd', '2nd'], 
        'name': ['Miller', 'Jacobson', 'Ali', 'Milner', 'Cooze', 'Jacon', 'Ryaner', 'Sone', 'Sloan', 'Piger', 'Riani', 'Ali'], 
        'preTestScore': [4, 24, 31, 2, 3, 4, 24, 31, 2, 3, 2, 3],
        'postTestScore': [25, 94, 57, 62, 70, 25, 94, 57, 62, 70, 62, 70]}



- a. Cree un dataframe con nombre 'regiment'. No olvide asignar el nombre correspondiente a cada columna.
- b. Muestre un resumen estadístico para cada compañia
- c. Muestre la media de preTestScores agrupado por regiment y company
- d. Muestre la media de preTestScores agrupado por regiment y company sin usar un índice jerárquico, es decir, que cada company represente una columna.
- e. ¿Cúal es el número de observaciones por regiment y company?
- f. Agrupe sobre regiment, itere sobre los grupos creados e imprima la información de cada uno.





**a.** Cree un dataframe con nombre 'regiment'. No olvide asignar el nombre correspondiente a cada columna.

In [0]:
# Respuesta aqui

In [0]:
# Respuesta aqui



**b.** Muestre un resumen estadístico para cada compañia

In [0]:
# Respuesta aqui



**c.** Muestre la media de preTestScores agrupado por regiment y company

In [0]:
# Respuesta aqui



**d.** Muestre la media de preTestScores agrupado por regiment y company sin usar un índice jerárquico, es decir, que cada company represente una columna.

In [0]:
# Respuesta aqui



**e.** ¿Cúal es el número de observaciones por regiment y company?

In [0]:
# Respuesta aqui



**f.** Agrupe sobre regiment, itere sobre los grupos creados e imprima la información de cada uno.

In [0]:
# Respuesta aqui